In [1]:
import torch
import numpy as np
from torch import Tensor
from numpy import ndarray as array
from helper import to_2d, tensor_size

from typing import Callable, Dict, Tuple, List, NamedTuple

# Boston data

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()

In [4]:
data = boston.data
target = boston.target
features = boston.feature_names

## SciKit Learn Linear Regression

In [5]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

In [6]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept=True)
lr.fit(data, target)
lr.score(data, target)

0.7406077428649427

In [7]:
import numpy as np
np.round(lr.coef_, 2)

array([-0.92,  1.08,  0.14,  0.68, -2.06,  2.67,  0.02, -3.1 ,  2.66,
       -2.08, -2.06,  0.86, -3.75])

### Manual linear regression

In [13]:
data, target = Tensor(data), Tensor(target)

In [14]:
def loss_gradients(forward_info) -> Tensor:
    
    dLdP = -(forward_info['y'] - forward_info['P'])
    
    dPdN = torch.ones_like(forward_info['N'])
    
    dLdN = dLdP * dPdN
    
    dNdW = forward_info['X'].transpose(0, 1)

    dLdW = torch.mm(dNdW, dLdN)
    
    dPdB = torch.ones_like(forward_info['B'])
    
    dLdB = dLdP * dPdB
    
    return dLdW, dLdB

In [22]:
from typing import Iterator
Batch = Tuple[Tensor, Tensor]

def generate_batch(X: Tensor, 
                   y: Tensor,
                   start: int = 0,
                   batch_size: int = 10) -> Iterator[Batch]:
    
    assert (X.dim() == 2) and (y.dim() == 2), \
    "X and Y must be 2 dimensional"

    if start+batch_size > X.shape[0]:
        batch_size = X.shape[0] - start
    
    X_batch, y_batch = X[start:start+batch_size], y[start:start+batch_size]
    
    return X_batch, y_batch

In [23]:
def forward_loss(X: Tensor,
                 y: Tensor,
                 W: Tensor,
                 B: Tensor) -> Tuple[Dict[str, Tensor], float]:

    # For the matrix multiplication to work,
    assert X.shape[1] == W.shape[0], \
    "Dimensions of betas and feature size do not match"

    N = torch.mm(X, W)

    P = torch.add(N, B.item())

    loss = torch.sum(torch.pow(y - P, 2)).item()

    forward_info: Dict[str, Tensor] = {}
    forward_info['X'] = X
    forward_info['W'] = W
    forward_info['B'] = B
    forward_info['N'] = N
    forward_info['P'] = P
    forward_info['y'] = y

    return forward_info, loss

In [24]:
def train(X: Tensor, 
          y: Tensor, 
          n_iter: int = 1000,
          learning_rate: float = 0.01,
          batch_size: int = 100,
          return_losses: bool = False, 
          return_weights: bool = False) -> None:

    y = to_2d(y, "col")
    start = 0

    # Initialize weights
    W = torch.empty(X.shape[1], 1).uniform_(-1, 1)
    B = torch.empty(1, 1).uniform_(-1, 1)

    if return_losses:
        losses = []

    for i in range(n_iter):

        if start >= X.shape[0]:
            X, y = permute_data(X, y)
            start = 0
        
        X_batch, y_batch = generate_batch(X, y, start, batch_size)

        start += batch_size
    
        forward_info, loss = forward_loss(X_batch, y_batch, W, B)

        if return_losses:
            losses.append(loss)

        dLdW, dLdB = loss_gradients(forward_info)
        W -= learning_rate * dLdW
        B -= learning_rate * torch.sum(dLdB)
    
    if return_weights:
        weights: Dict[str, Tensor] = {}
        weights['W'] = W
        weights['B'] = B
        return losses, weights
    
    return losses

In [25]:
torch.manual_seed(80718)
train_info = train(data, target, 
                   learning_rate = 0.001,
                   batch_size=23, 
                   return_losses=True, 
                   return_weights=True)
losses = train_info[0]
weights = train_info[1]

In [26]:
def predict(X: Tensor, 
            y: Tensor, 
            weights: Dict[str, Tensor]):
    
    N = torch.mm(X, weights['W'])

    return torch.add(N, weights['B'].item())

In [27]:
preds = predict(data, target, weights)

In [28]:
from sklearn.metrics import r2_score
r2_score(preds, target)

0.6174701879681399

### Tuning learning rate

In [29]:
def r2_score_2(learning_rate: float = 0.01, 
             n_iter: int = 1000) -> float:
    torch.manual_seed(80718)
    train_info = train(data, target, 
                       learning_rate=learning_rate,
                       batch_size=23, 
                       n_iter=n_iter,
                       return_losses=True, 
                       return_weights=True)

    losses = train_info[0]
    weights = train_info[1]

    preds = predict(data, target, weights)
    
    return r2_score(preds, target)

In [30]:
lrs = np.geomspace(0.01, 0.00001, 50)

In [31]:
r2s = [r2_score_2(float(lr), 10000) for lr in lrs]

In [32]:
plt.xlim(lrs[-1], lrs[0])
plt.semilogx(lrs, r2s)

NameError: name 'plt' is not defined

### Analyzing best model

In [ ]:
torch.manual_seed(80718)
train_info = train(data, target, 
                   learning_rate=0.0001,
                   batch_size=23, 
                   n_iter=10000,
                   return_losses=True, 
                   return_weights=True)

losses = train_info[0]
weights = train_info[1]

preds = predict(data, target, weights)

In [ ]:
weights['W']

In [ ]:
lr.coef_

In [ ]:
weights['B']

In [ ]:
lr.intercept_

## Plotting loss

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.semilogy(list(range(10000)), losses) 

# Logistic regression

### Helpers

In [ ]:
for obs in range(a.shape[0]): # for each observation in the batch
    exp_obs = torch.exp(a[obs])
    sum_exp_obs = exp_obs.sum().item()
    softmax_obs = exp_obs / sum_exp_obs
    print(softmax_obs)

In [ ]:
def sigmoid(x: Tensor, 
            deriv: bool=False) -> Tensor:
    if deriv:
        return sigmoid(x) * (1 - sigmoid(x))
    else:
        return 1 / (1 + torch.exp(-1.0 * x))

In [ ]:
def softmax(x: Tensor) -> Tensor:

    assert x.dim() == 2, \
    "Expect Tensor with shape (batch_size, num_classes), instead " + \
    "x has shape {0}".format(x.shape)
    
    def _softmax_row(row: Tensor) -> Tensor:
        
        assert row.dim() == 1, \
        "'row' should indeed be a row, instead it has shape" \
        .format(row.shape)
        
        exp_obs = torch.exp(row)
        sum_exp_obs = exp_obs.sum().item()
        softmax_obs = exp_obs / sum_exp_obs
        
        return softmax_obs

    output_rows = []
    for obs in range(x.shape[0]):
        output_row = to_2d(_softmax_row(x[obs]), "row")
        output_rows.append(output_row)
        
    return torch.cat(output_rows)
    

In [ ]:
softmax(Tensor([[10, 8, 6, 4, 2]]))

In [ ]:
a.sum()

## Sklearn logistic regression

In [33]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target
features = breast_cancer.feature_names

### Standardize X

In [48]:
s = StandardScaler()
data = s.fit_transform(data)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [41]:
logr = LogisticRegression(C=10e9)
logr.fit(data, target)
preds = logr.predict(data)

In [42]:
print("{:.3f}".format(accuracy_score(preds, target)))

0.965


In [43]:
print("{:.3f}".format(f1_score(preds, target)))

0.972


### Hand rolled logistic regression

#### Data preprocessing

In [44]:
def predictions_to_2d(predictions: Tensor) -> Tensor:
    
    assert predictions.shape[1] == 1, \
    "Expected a column for predictions, got shape: {}".format(predictions.shape)
    
    inverse_predictions = 1 - predictions
    
    return torch.cat([predictions, inverse_predictions], dim=1)

In [45]:
data_tensor, target_tensor = Tensor(data), Tensor(to_2d(Tensor(target), "col"))

In [46]:
target_tensor = predictions_to_2d(target_tensor)

In [ ]:
data_standard = standardize_data(data)
data_standard = Tensor(data_standard)

### Modeling functions

In [ ]:
def forward_logistic(observations: Tensor,
                     betas: Tensor) -> Tensor:
    
    # For the matrix multiplication to work, 
    assert observations.shape[1] == betas.shape[0], \
    "Dimensions of betas and feature size do not match"
    
    mult = torch.mm(observations, betas)
    predictions = sigmoid(mult)
    predictions = predictions_to_2d(predictions)
    softmax_preds = softmax(predictions)
    
    forward_info: Dict[str, Tensor] = {}
    forward_info['betas'] = betas
    forward_info['observations'] = observations
    forward_info['mult'] = mult
    forward_info['predictions'] = predictions   
    forward_info['softmax'] = softmax_preds   

    return forward_info

In [ ]:
def cross_entropy(predictions: Tensor, 
                  actual: Tensor) -> Tensor:
    
    assert predictions.shape == actual.shape, \
    "Prediction and actual must have same shape"
    
    return -1.0 * actual * torch.log(predictions) - (1.0 - actual) * torch.log(1 - predictions)

In [ ]:
def loss(forward_info: Dict[str, Tensor], 
         actual: Tensor, 
         kind: str = "binary_crossentropy") -> Tensor:
     
    assert kind in ("binary_crossentropy"), \
    "Inappropriate loss type"

    predictions = forward_info['softmax']

    n = predictions.shape[0]
    
    ce = cross_entropy(predictions, actual)

    return ce.sum().item() / n

In [ ]:
def loss_bce_softmax_deriv(predictions: Tensor, 
                           actual: Tensor, 
                           kind: str = "binary_crossentropy", 
                           softmax: bool=True) -> Tensor:
     
    assert kind in ("binary_crossentropy"), \
    "Inappropriate loss type"

    assert predictions.shape == actual.shape, \
    "Prediction and actual must have same shape"

    return to_2d((predictions - actual)[:, 0], "col")

In [ ]:
def beta_update_logistic(forward_info: Dict[str, Tensor], 
                         actual: Tensor, 
                         softmax: bool=True) -> Tensor:

    obs = forward_info['observations'] 
    predictions = forward_info['softmax']        

    n = obs.shape[0]

    dLdP = loss_bce_softmax_deriv(predictions, actual)
    dPdB = sigmoid(forward_info['mult'], deriv=True)
    dLdB = dPdB * dLdP
    
    dBdA = obs.transpose(0, 1)

    return dBdA.mm(dLdB)

In [ ]:
def train_model_logistic(data: Tensor, 
                         target: Tensor, 
                         learning_rate: float = 0.01, 
                         num_iter: int = int(1e3)) -> Tensor:
    
    # Generate random betas
    torch.manual_seed(61818)
    betas = torch.randn((data.shape[1], 1))

    # For each iteration
    for i in range(int(num_iter)):
        # Generate a random batch
        np.random.seed(61818)
        x_batch, y_batch = generate_batch(data, target, 100)

        # Forward through the network
        forward_info = forward_logistic(x_batch, betas)
        L = loss(forward_info, y_batch)
        if i % int(1e2) == 0:
            print("Loss after {:d} iterations: {:.3f}".
                  format(i, L))

        # Compute the beta update
        update = beta_update_logistic(forward_info, y_batch)
        
        # Update the betas
        betas = betas - learning_rate * update 
    return betas

In [ ]:
beta_pred = train_model_logistic(data_standard, target_tensor, 
                                 num_iter=500)

### Prediction

In [ ]:
pred_info = forward_logistic(data_standard, beta_pred)

In [ ]:
predictions = pred_info['predictions'] > 0.5

In [ ]:
accuracy_score(predictions[:, 0], target)

In [ ]:
f1_score(predictions[:, 0], target)

The point of this exercise was just to sanity check that our math was correct; that these operations, which we argued through a bunch of equations would lead to us solving a logistic regression problem, actually does lead to that.